# **Forest Fire Prediction**

### Part 3: Gathering data from wether API

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

pd.options.mode.chained_assignment = None

In [ ]:
API_URL = 'https://archive-api.open-meteo.com/v1/archive'

FINAL_CSV = 'fire_history_final.csv'

PARAMETERS = ['temperature_2m_max', 'temperature_2m_min', 'windspeed_10m_max', 'winddirection_10m_dominant', 'shortwave_radiation_sum', 'precipitation_sum']
WEATHER_COLS = ['MaxTemperature', 'MinTemperature', 'WindSpeed', 'WindDirection', 'Radiation', 'Precipitation']


In [ ]:
def get_weather(latitude, longitude, date):
    params = {'latitude': latitude, 'longitude': longitude, 'start_date': date, 'end_date': date, 
              'daily': PARAMETERS, 'mim': date, 'max': date, 'timezone': 'GMT'}
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        return response.json()['daily']
    else:
        return None

In [ ]:
def add_weather_data(df):
    df_updated = df.copy()
    df_updated[WEATHER_COLS] = np.nan

    lats = df_updated['InitialLatitude'].tolist()
    longs = df_updated['InitialLongitude'].tolist()
    dates = df_updated['FireDiscoveryDateTime'].tolist()

    final_data = []
    final_data = [get_weather(lat, lon, date) for lat, lon, date in zip(lats, longs, dates)]

    for row in df_updated.index:
        for i in range(len(WEATHER_COLS)):
            df_updated[WEATHER_COLS[i]][row] = final_data[row][PARAMETERS[i]][0]

    return df_updated

In [ ]:
df.to_csv(FINAL_CSV, index=False)
df